In [2]:
import itertools
from transformers import pipeline
import torch

/Users/dominik/Desktop/University/Semestr5/ModeleJezykowe/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
polka = pipeline("text-generation", model="eryk-mazus/polka-1.1b")
model = polka.model 
tokenizer = polka.tokenizer

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
def score_sequence(prompt, continuation):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    continuation_ids = tokenizer.encode(continuation, return_tensors='pt')

    input_ids_full = torch.cat([input_ids, continuation_ids[:, 1:]], dim=-1)

    with torch.no_grad():
        outputs = model(input_ids_full, labels=input_ids_full)
        log_likelihood = -outputs.loss * continuation_ids.size(1)

    return log_likelihood.item()

def classify_opinion(opinion):
    methods = []

    prompt = opinion.strip()
    continuation_pos = " To jest pozytywna opinia."
    continuation_neg = " To jest negatywna opinia."

    score_pos = score_sequence(prompt, continuation_pos)
    score_neg = score_sequence(prompt, continuation_neg)

    result1 = "Pozytywna" if score_pos > score_neg else "Negatywna"
    methods.append(result1)

    prompt = opinion.strip() + " Ogólnie rzecz biorąc, jest to opinia"
    continuation_pos = " pozytywna."
    continuation_neg = " negatywna."

    score_pos = score_sequence(prompt, continuation_pos)
    score_neg = score_sequence(prompt, continuation_neg)

    result2 = "Pozytywna" if score_pos > score_neg else "Negatywna"
    methods.append(result2)

    final_result = "Pozytywna" if methods.count("Pozytywna") > methods.count("Negatywna") else "Negatywna"

    return final_result

In [5]:
positive_opinions = [
    "Parking monitorowany w cenie.",
    "Hotel czysty, pokoje były sprzątane bardzo dokładnie.",
    "Generalnie mogę go polecić, kierował mnie na potrzebne badania, analizował ich wyniki, cierpliwie odpowiadał na pytania.",
    "Fajny klimat pofabrykanckich kamienic.",
    "Sala zabaw dla dzieci, plac zabaw na zewnątrz, kominek, tenis stołowy."
]

negative_opinions = [
    "W wielu pokojach niedziałająca klimatyzacja.",
    "Jedzenie mimo rzekomych dni europejskich monotonne.",
    "Drożej niż u konkurencji w podobnym standardzie.",
    "Może za szybko zrezygnowałam, ale szkoda mi było wydawać pieniędzy na spotkania, które nie przynosiły efektu.",
    "Omijaj to miejsce!"
]

correct = 0
total = 0

print("Positive Opinions:")
for opinion in positive_opinions:
    prediction = classify_opinion(opinion)
    print(f"Opinion: {opinion}\nPredicted Sentiment: {prediction}\n")
    if prediction == "Pozytywna":
        correct += 1
    total += 1

print("Negative Opinions:")
for opinion in negative_opinions:
    prediction = classify_opinion(opinion)
    print(f"Opinion: {opinion}\nPredicted Sentiment: {prediction}\n")
    if prediction == "Negatywna":
        correct += 1
    total += 1

accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}%")

Positive Opinions:
Opinion: Parking monitorowany w cenie.
Predicted Sentiment: Negatywna

Opinion: Hotel czysty, pokoje były sprzątane bardzo dokładnie.
Predicted Sentiment: Pozytywna

Opinion: Generalnie mogę go polecić, kierował mnie na potrzebne badania, analizował ich wyniki, cierpliwie odpowiadał na pytania.
Predicted Sentiment: Pozytywna

Opinion: Fajny klimat pofabrykanckich kamienic.
Predicted Sentiment: Pozytywna

Opinion: Sala zabaw dla dzieci, plac zabaw na zewnątrz, kominek, tenis stołowy.
Predicted Sentiment: Pozytywna

Negative Opinions:
Opinion: W wielu pokojach niedziałająca klimatyzacja.
Predicted Sentiment: Negatywna

Opinion: Jedzenie mimo rzekomych dni europejskich monotonne.
Predicted Sentiment: Negatywna

Opinion: Drożej niż u konkurencji w podobnym standardzie.
Predicted Sentiment: Negatywna

Opinion: Może za szybko zrezygnowałam, ale szkoda mi było wydawać pieniędzy na spotkania, które nie przynosiły efektu.
Predicted Sentiment: Negatywna

Opinion: Omijaj to mie

In [7]:
def classify_from_file(filename, maximum=None):
    correct = 0
    total = 0
    
    with open(filename, 'r', encoding='utf-8') as file:
        max_iter = 0
        for line in file:
            if(maximum != None):
                if(max_iter >= maximum):
                    break
                max_iter += 1
            if line.startswith("GOOD"):
                label = "Pozytywna"
                opinion = line[len("GOOD"):].strip()
            elif line.startswith("BAD"):
                label = "Negatywna"
                opinion = line[len("BAD"):].strip()
            else:
                continue  

            prediction = classify_opinion(opinion)
            if prediction == label:
                correct += 1
            total += 1
    
    accuracy = correct / total * 100 if total > 0 else 0
    print(f"Accuracy on file data: {accuracy:.2f}%")
    return accuracy

filename = "reviews_for_task3.txt"
classify_from_file(filename, 100)

Accuracy on file data: 76.00%


76.0